In [15]:
using QuantumOptics
using OrdinaryDiffEq
using ModelingToolkit
using LinearAlgebra
using Symbolics
using SymbolicUtils
using DifferentialEquations

In [16]:
include("../src/indexing.jl")
include("../src/doubleSums.jl")
include("../src/averageSums.jl")
include("../src/indexedMeanfield.jl")

isNotIn (generic function with 1 method)

In [17]:
N = :N
@cnumbers Δ

#Define Hilberspaces, one for atom, one for field
ha = NLevelSpace(:a,2)
hf= FockSpace(:field)
h = tensor(hf,ha)

#define operators
a = Destroy(h,:a)
σ(i,j) = Transition(h,:σ,i,j)

σ (generic function with 1 method)

In [18]:
#define different indices
j_ind = Index(h,:j,N)
k_ind = Index(h,:k,N)
l_ind = Index(h,:l,N)

#define a indexed varaible: g with index j: gⱼ
g_j = IndexedVariable(:g,j_ind)

#define some operators:

#with Index j
σ12_j = IndexedOperator(σ(1,2),j_ind)
σ21_j = IndexedOperator(σ(2,1),j_ind)

#with Index k
σ12_k = IndexedOperator(σ(1,2),k_ind)
σ21_k = IndexedOperator(σ(2,1),k_ind)
σ22_k = IndexedOperator(σ(2,2),k_ind)

#with Index l
σ12_l = IndexedOperator(σ(1,2),l_ind)

σ12l

In [19]:
#Define Sums
sum1 = IndexedSingleSum(g_j*σ12_j*a',j_ind)
sum2 = IndexedSingleSum(g_j*σ21_j*a,j_ind)

#can also be done in one step as:
IndexedSum = IndexedSingleSum((g_j*σ12_j*a' + g_j*σ21_j*a),j_ind)

#Hamiltonian:
H = -Δ*a'*a + IndexedSum

(Σ(j=1:N)gj*(a′*σ12j)+Σ(j=1:N)gj*(a*σ21j)+-Δ*(a′*a))

In [20]:
sum1+sum2

(Σ(j=1:N)gj*(a′*σ12j)+Σ(j=1:N)gj*(a*σ21j))

In [21]:
#Example 1: calculate commutators
simplify(commutator(H,a'*σ12_k))

(Σ(j=1:N)(j≠k)gj*(σ21j*σ12k)+gk*(σ22k)+-gk*(a′*a)+2gk*(a′*a*σ22k)+-Δ*(a′*σ12k))

In [22]:
simplify(commutator(H,σ22_k))

(gk*(a′*σ12k)+-gk*(a*σ21k))

In [23]:
#Example: calculate Meanfield-equations for the k-th atom
ops = [σ22_k]
J = [σ12_k]
rates = 1.0

de = meanfield(ops,H,J;rates=rates,order=2)

∂ₜ(⟨σ22k⟩) = (0 + 1im)*gk*⟨a′*σ12k⟩ + (0 - 1im)*gk*⟨a*σ21k⟩ - ⟨σ22k⟩


In [24]:
#complete equations in first order
eqs_com = indexedComplete(de;order=1)

∂ₜ(⟨σ22k⟩) = (0 + 1im)*gk*⟨a′⟩*⟨σ12k⟩ + (0 - 1im)*gk*⟨a⟩*⟨σ21k⟩ - ⟨σ22k⟩
∂ₜ(⟨a′⟩) = (0 + 1im)*var"∑(j=1:N)gj*⟨σ21j⟩" + (0 - 1im)*Δ*⟨a′⟩
∂ₜ(⟨σ12k⟩) = (0 - 1im)*gk*⟨a⟩ + (0 + 2im)*gk*⟨a⟩*⟨σ22k⟩ - 0.5⟨σ12k⟩


In [25]:
#complete equations in second order
eqs_com = indexedComplete(de;order=2)

∂ₜ(⟨σ22k⟩) = (0 + 1im)*gk*⟨a′*σ12k⟩ + (0 - 1im)*gk*⟨a*σ21k⟩ - ⟨σ22k⟩
∂ₜ(⟨a′*σ12k⟩) = (0 + 1im)*var"∑(j=1:N)(j≠k)gj*⟨σ21j*σ12k⟩" + (0 + 1im)*gk*⟨σ22k⟩ + (0 - 1im)*gk*⟨a′*a⟩ + (0 - 1im)*Δ*⟨a′*σ12k⟩ + (0 + 2im)*gk*(⟨σ22k⟩*⟨a′*a⟩ + ⟨a⟩*⟨a′*σ22k⟩ + ⟨a′⟩*⟨a*σ22k⟩ - 2⟨a′⟩*⟨a⟩*⟨σ22k⟩) - 0.5⟨a′*σ12k⟩
∂ₜ(⟨a′*a⟩) = (0 + 1im)*var"∑(j=1:N)gj*⟨a*σ21j⟩" + (0 - 1im)*var"∑(j=1:N)gj*⟨a′*σ12j⟩"
∂ₜ(⟨a⟩) = (0 - 1im)*var"∑(j=1:N)gj*⟨σ12j⟩" + (0 + 1im)*Δ*⟨a⟩
∂ₜ(⟨a′*σ22k⟩) = (0 + 1im)*var"∑(j=1:N)(j≠k)gj*⟨σ21j*σ22k⟩" + (0 + 1im)*gk*(⟨σ12k⟩*⟨a′*a′⟩ + 2⟨a′⟩*⟨a′*σ12k⟩ - 2(⟨a′⟩^2)*⟨σ12k⟩) + (0 - 1im)*gk*(⟨a⟩*⟨a′*σ21k⟩ + ⟨a′⟩*⟨a*σ21k⟩ + ⟨σ21k⟩*⟨a′*a⟩ - 2⟨a′⟩*⟨a⟩*⟨σ21k⟩) + (0 - 1im)*Δ*⟨a′*σ22k⟩ - ⟨a′*σ22k⟩
∂ₜ(⟨σ12k*σ21q⟩) = var"(k≠q)(0 - 1im)*gk*⟨a*σ21q⟩" + var"(k≠q)(0 - 1im)*gq*(⟨σ12k⟩*⟨a′*σ22q⟩ + ⟨σ22q⟩*⟨a′*σ12k⟩ + ⟨a′⟩*⟨σ12k*σ22q⟩ - 2⟨a′⟩*⟨σ12k⟩*⟨σ22q⟩)" + var"(k≠q)-0.5⟨σ12k*σ21q⟩" + var"(q≠k)(0 + 1im)*gq*⟨a′*σ12k⟩" + var"(q≠k)(0 - 1im)*gq*(⟨σ12k⟩*⟨a′*σ22q⟩ + ⟨σ22q⟩*⟨a′*σ12k⟩ + ⟨a′⟩*⟨σ12k*σ22q⟩ - 2⟨a′⟩*⟨σ12k⟩*⟨σ

In [26]:
mapping = Dict{Symbol,Int64}((N=>10))

Dict{Symbol, Int64} with 1 entry:
  :N => 10

In [27]:
eqs_ = evalME(eqs_com;mapping)

∂ₜ(⟨σ221⟩) = (0 + 1im)*g1*⟨a′*σ121⟩ + (0 - 1im)*g1*⟨a*σ211⟩ - ⟨σ221⟩
∂ₜ(⟨σ222⟩) = (0 + 1im)*g2*⟨a′*σ122⟩ + (0 - 1im)*g2*⟨a*σ212⟩ - ⟨σ222⟩
∂ₜ(⟨σ223⟩) = (0 + 1im)*g3*⟨a′*σ123⟩ + (0 - 1im)*g3*⟨a*σ213⟩ - ⟨σ223⟩
∂ₜ(⟨σ224⟩) = (0 + 1im)*g4*⟨a′*σ124⟩ + (0 - 1im)*g4*⟨a*σ214⟩ - ⟨σ224⟩
∂ₜ(⟨σ225⟩) = (0 + 1im)*g5*⟨a′*σ125⟩ + (0 - 1im)*g5*⟨a*σ215⟩ - ⟨σ225⟩
∂ₜ(⟨σ226⟩) = (0 + 1im)*g6*⟨a′*σ126⟩ + (0 - 1im)*g6*⟨a*σ216⟩ - ⟨σ226⟩
∂ₜ(⟨σ227⟩) = (0 + 1im)*g7*⟨a′*σ127⟩ + (0 - 1im)*g7*⟨a*σ217⟩ - ⟨σ227⟩
∂ₜ(⟨σ228⟩) = (0 + 1im)*g8*⟨a′*σ128⟩ + (0 - 1im)*g8*⟨a*σ218⟩ - ⟨σ228⟩
∂ₜ(⟨σ229⟩) = (0 + 1im)*g9*⟨a′*σ129⟩ + (0 - 1im)*g9*⟨a*σ219⟩ - ⟨σ229⟩
∂ₜ(⟨σ2210⟩) = (0 + 1im)*g10*⟨a′*σ1210⟩ + (0 - 1im)*g10*⟨a*σ2110⟩ - ⟨σ2210⟩
∂ₜ(⟨a′*σ121⟩) = (0 + 1im)*g1*⟨σ221⟩ + (0 + 2im)*g1*(⟨a′⟩*⟨a*σ221⟩ + ⟨σ221⟩*⟨a′*a⟩ + ⟨a⟩*⟨a′*σ221⟩ - 2⟨a′⟩*⟨a⟩*⟨σ221⟩) + (0 - 1im)*g1*⟨a′*a⟩ + (0 + 1im)*g10*⟨σ121*σ2110⟩ + (0 + 1im)*g5*⟨σ121*σ215⟩ + (0 + 1im)*g6*⟨σ121*σ216⟩ + (0 + 1im)*g9*⟨σ121*σ219⟩ + (0 + 1im)*g2*⟨σ121*σ212⟩ + (0 + 1im)*g3*⟨σ121*σ213⟩ 

In [28]:
#Double Indexed Sums
i_ind = Index(h,:i,N)
σ12_i = IndexedOperator(σ(1,2),i_ind)
ops = [σ22_k]
J = [σ12_i,σ12_j]
Jdagger = adjoint.(J)

#define double indexed Variable
rates_ind = DoubleIndexedVariable(:Γ,i_ind,j_ind)

LoadError: MethodError: no method matching DoubleIndexedVariable(::Symbol, ::Index, ::Index)
[0mClosest candidates are:
[0m  DoubleIndexedVariable(::Any, ::Any, ::Any, [91m::Any[39m) at e:\Google Drive\Uni\MSC\Arbeit\Indexing_Backup\current\QuantumCumulants.jl\src\indexing.jl:73

In [ ]:
IndexedSingleSum(σ21_k*σ21_j*σ12_i,i_ind)

(Σ(i=1:20)(i≠k,j)(σ12i*σ21k*σ21j)+(σ21k*σ21j*σ12k)+(σ21k*σ22j))

In [ ]:
sum1 = IndexedSingleSum(g_j*σ12_j*a',j_ind)

Σ(j=1:20)gj*(a′*σ12j)

In [ ]:
IndexedDoubleSum(sum1*σ21_k, k_ind, Index[])

(Σ(k=1:20)gk*(a′)+Σ(k=1:20)-gk*(a′*σ22k)+Σ(k=1:20)Σ(j=1:20)(j≠k)gj*(a′*σ12j*σ21k))

In [ ]:
#example: calculate diffusal terms in meanfield
#case when a Matrix of rates is given assosiated with the σᵢ and σⱼ
simplify(indexed_master_lindblad(ops[1],J,Jdagger,rates_ind))

LoadError: BoundsError: attempt to access 1-element Vector{Any} at index [3]